In [1]:
import torch

In [2]:
# Проверяем, доступен ли GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

In [1]:
from pyannote.audio import Pipeline
# Загрузка предобученной модели для диаризации
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token="")
pipeline.to(torch.device("cuda"))

# Применение диаризации на аудио файле
diarization = pipeline("audio.wav")

# Выводим результат диаризации
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"Speaker {speaker}: from {turn.start:.1f}s to {turn.end:.1f}s")


Could not download 'pyannote/speaker-diarization-3.1' pipeline.
It might be because the pipeline is private or gated so make
sure to authenticate. Visit https://hf.co/settings/tokens to
create your access token and retry with:

   >>> Pipeline.from_pretrained('pyannote/speaker-diarization-3.1',
   ...                          use_auth_token=YOUR_AUTH_TOKEN)

If this still does not work, it might be because the pipeline is gated:
visit https://hf.co/pyannote/speaker-diarization-3.1 to accept the user conditions.


AttributeError: 'NoneType' object has no attribute 'to'

In [8]:
diarization

NameError: name 'diarization' is not defined

In [2]:
import whisper
# Загружаем модель Whisper
model = whisper.load_model("medium", device=device)

ModuleNotFoundError: No module named 'whisper'

In [3]:

# Распознаем речь с аудио файла
result = model.transcribe("audio.wav")

# Выводим 100 символов результата
print(result['text'][:100])

NameError: name 'model' is not defined

In [4]:
# Список для хранения результата
dialogue = []

# Проходим по каждому сегменту, который вернул Whisper
for segment in result['segments']:
    # Для каждого сегмента ищем говорящего в диаризации
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        if turn.start <= segment['start'] <= turn.end:
            # Если последний спикер такой же, как текущий, объединяем текст
            if dialogue and dialogue[-1][0] == speaker:
                dialogue[-1][1] += " " + segment['text']  # Объединяем текст
            else:
                dialogue.append([speaker, segment['text']])  # Добавляем новый сегмент
            break

# Выводим результат
for entry in dialogue:
    print(f"Speaker {entry[0]}: {entry[1]}")

NameError: name 'result' is not defined

In [5]:
import re

# Функция для поиска последнего знака препинания в тексте
def find_last_punctuation(text):
    match = re.search(r'[.!?](?!.*[.!?])', text)  # Ищем последний знак препинания
    if match:
        return match.end()
    return None

# Проходим по диалогу и перераспределяем текст
i = 0
while i < len(dialogue) - 1:
    speaker, text = dialogue[i]
    
    # Если текст не заканчивается на . ! или ?
    if not re.search(r'[.!?]$', text):
        # Находим позицию последнего знака препинания
        last_punctuation_pos = find_last_punctuation(text)
        
        if last_punctuation_pos:
            # Разделяем текст на две части
            main_text = text[:last_punctuation_pos]
            leftover_text = text[last_punctuation_pos:].strip()
            
            # Обновляем текущий текст с правильным окончанием
            dialogue[i][1] = main_text
            
            # Добавляем оставшийся текст к следующему спикеру

            dialogue[i + 1][1] = leftover_text + dialogue[i + 1][1]
    
    i += 1

# Выводим результат
for speaker, text in dialogue:
    print(f"Speaker {speaker}: {text}")

In [6]:
# Функция для очистки текста от лишних пробелов
def clean_text(text):
    # Удаляем лишние пробелы в начале и конце строки
    text = text.strip()
    # Заменяем последовательности из двух или более пробелов на один пробел
    text = re.sub(r'\s{2,}', ' ', text)
    return text

# Проходим по каждому сегменту диалога и очищаем текст
for i in range(len(dialogue)):
    speaker, text = dialogue[i]
    dialogue[i][1] = clean_text(text)

# Выводим результат
for speaker, text in dialogue:
    print(f"{speaker}: {text}")


Сохраняем

In [7]:
# Функция для сохранения диалога в текстовый файл
def save_dialogue_to_file(dialogue, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for speaker, text in dialogue:
            file.write(f"{speaker}: {text}\n")

# Сохраняем диалог в файл
save_dialogue_to_file(dialogue, 'dialogue.txt')

print("Диалог сохранен в файл 'dialogue.txt'.")

Диалог сохранен в файл 'dialogue.txt'.
